In [1]:
import datetime
import json
import logging
import os

from xcube.core.store import new_data_store
from xcube.core.chunk import chunk_dataset
import numpy as np
import dask.array as da
from zappend.api import zappend


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

LOG = logging.getLogger("MyLogger")

In [2]:
with open("cds-credentials.json") as f:
    credentials = json.load(f)

cds_store = new_data_store(
    "cds",
    **credentials,
    normalize_names=True,
)

In [3]:
store_team = new_data_store(
    "s3",
    root=os.environ["S3_USER_STORAGE_BUCKET"],
    max_depth=4,
    storage_options=dict(
        anon=False,
        key=os.environ["S3_USER_STORAGE_KEY"],
        secret=os.environ["S3_USER_STORAGE_SECRET"],
    ),
)

In [4]:
def get_time_ranges():
    time_ranges = []
    start = datetime.date.fromisoformat("2016-11-01")
    end = datetime.date.fromisoformat("2016-12-31")
    count_start = start
    while count_start < end:
        count_end = count_start + datetime.timedelta(days=1)
        time_ranges.append([count_start.isoformat(), count_end.isoformat()])
        count_start = count_end + datetime.timedelta(days=1)
    time_ranges[-1][1] = end.isoformat()
    
    for year in range(2017, 2025):
        start = datetime.date.fromisoformat(f"{year}-01-01")
        end = datetime.date.fromisoformat(f"{year}-12-31")
        count_start = start
        while count_start < end:
            count_end = count_start + datetime.timedelta(days=1)
            time_ranges.append([count_start.isoformat(), count_end.isoformat()])
            count_start = count_end + datetime.timedelta(days=1)
        time_ranges[-1][1] = end.isoformat()

    return time_ranges

time_ranges = get_time_ranges()
time_ranges

[['2016-11-01', '2016-11-02'],
 ['2016-11-03', '2016-11-04'],
 ['2016-11-05', '2016-11-06'],
 ['2016-11-07', '2016-11-08'],
 ['2016-11-09', '2016-11-10'],
 ['2016-11-11', '2016-11-12'],
 ['2016-11-13', '2016-11-14'],
 ['2016-11-15', '2016-11-16'],
 ['2016-11-17', '2016-11-18'],
 ['2016-11-19', '2016-11-20'],
 ['2016-11-21', '2016-11-22'],
 ['2016-11-23', '2016-11-24'],
 ['2016-11-25', '2016-11-26'],
 ['2016-11-27', '2016-11-28'],
 ['2016-11-29', '2016-11-30'],
 ['2016-12-01', '2016-12-02'],
 ['2016-12-03', '2016-12-04'],
 ['2016-12-05', '2016-12-06'],
 ['2016-12-07', '2016-12-08'],
 ['2016-12-09', '2016-12-10'],
 ['2016-12-11', '2016-12-12'],
 ['2016-12-13', '2016-12-14'],
 ['2016-12-15', '2016-12-16'],
 ['2016-12-17', '2016-12-18'],
 ['2016-12-19', '2016-12-20'],
 ['2016-12-21', '2016-12-22'],
 ['2016-12-23', '2016-12-24'],
 ['2016-12-25', '2016-12-26'],
 ['2016-12-27', '2016-12-28'],
 ['2016-12-29', '2016-12-31'],
 ['2017-01-01', '2017-01-02'],
 ['2017-01-03', '2017-01-04'],
 ['2017-

In [5]:
# extract ERA5 from CDS
bbox = [-5, 40, 35, 65]
variable_names = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4",
    "skin_reservoir_content",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4",
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards",
    "potential_evaporation",
    "runoff",
    "total_evaporation",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation",
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation"
]
for time_range in time_ranges:
    file_name = time_range[0].replace("-", "_")
    data_id = f"cubes/aux/era5_land/{file_name}.zarr"
    if store_team.has_data(data_id):
        print(f"ERS5 for {time_range} is already written to {data_id}")
        continue
    print(time_range)
    ds = cds_store.open_data(
        "reanalysis-era5-land",
        "dataset:netcdf:cds",
        variable_names=variable_names,
        bbox=bbox,
        spatial_res=0.1,
        time_range=time_range,
    )
    store_team.write_data(ds, data_id, replace=True)
    print(f"ERS5 for {time_range} is written to {data_id}")

ERS5 for ['2016-11-01', '2016-11-02'] is already written to cubes/aux/era5_land/2016_11_01.zarr
ERS5 for ['2016-11-03', '2016-11-04'] is already written to cubes/aux/era5_land/2016_11_03.zarr
ERS5 for ['2016-11-05', '2016-11-06'] is already written to cubes/aux/era5_land/2016_11_05.zarr
ERS5 for ['2016-11-07', '2016-11-08'] is already written to cubes/aux/era5_land/2016_11_07.zarr
ERS5 for ['2016-11-09', '2016-11-10'] is already written to cubes/aux/era5_land/2016_11_09.zarr
ERS5 for ['2016-11-11', '2016-11-12'] is already written to cubes/aux/era5_land/2016_11_11.zarr
ERS5 for ['2016-11-13', '2016-11-14'] is already written to cubes/aux/era5_land/2016_11_13.zarr
ERS5 for ['2016-11-15', '2016-11-16'] is already written to cubes/aux/era5_land/2016_11_15.zarr
ERS5 for ['2016-11-17', '2016-11-18'] is already written to cubes/aux/era5_land/2016_11_17.zarr
ERS5 for ['2016-11-19', '2016-11-20'] is already written to cubes/aux/era5_land/2016_11_19.zarr
ERS5 for ['2016-11-21', '2016-11-22'] is

xcube-cds version 1.0.0
2025-05-13 05:52:11,873 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 05:52:11,873 - INFO - [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 05:52:11,874 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 05:52:11,874 - WARNING - [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 05:52:12,096 INFO Request ID is ae4b95b3-9441-47e6-9616-922549e

ERS5 for ['2018-08-23', '2018-08-24'] is written to cubes/aux/era5_land/2018_08_23.zarr
ERS5 for ['2018-08-25', '2018-08-26'] is already written to cubes/aux/era5_land/2018_08_25.zarr
ERS5 for ['2018-08-27', '2018-08-28'] is already written to cubes/aux/era5_land/2018_08_27.zarr
ERS5 for ['2018-08-29', '2018-08-30'] is already written to cubes/aux/era5_land/2018_08_29.zarr
ERS5 for ['2018-08-31', '2018-09-01'] is already written to cubes/aux/era5_land/2018_08_31.zarr
ERS5 for ['2018-09-02', '2018-09-03'] is already written to cubes/aux/era5_land/2018_09_02.zarr
ERS5 for ['2018-09-04', '2018-09-05'] is already written to cubes/aux/era5_land/2018_09_04.zarr
ERS5 for ['2018-09-06', '2018-09-07'] is already written to cubes/aux/era5_land/2018_09_06.zarr
ERS5 for ['2018-09-08', '2018-09-09'] is already written to cubes/aux/era5_land/2018_09_08.zarr
ERS5 for ['2018-09-10', '2018-09-11'] is already written to cubes/aux/era5_land/2018_09_10.zarr
ERS5 for ['2018-09-12', '2018-09-13'] is already

xcube-cds version 1.0.0
2025-05-13 05:57:17,674 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 05:57:17,674 - INFO - [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 05:57:17,675 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 05:57:17,675 - WARNING - [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


['2019-01-13', '2019-01-14']


...

ERS5 for ['2019-02-18', '2019-02-19'] is already written to cubes/aux/era5_land/2019_02_18.zarr
ERS5 for ['2019-02-20', '2019-02-21'] is already written to cubes/aux/era5_land/2019_02_20.zarr
ERS5 for ['2019-02-22', '2019-02-23'] is already written to cubes/aux/era5_land/2019_02_22.zarr
ERS5 for ['2019-02-24', '2019-02-25'] is already written to cubes/aux/era5_land/2019_02_24.zarr
ERS5 for ['2019-02-26', '2019-02-27'] is already written to cubes/aux/era5_land/2019_02_26.zarr
['2019-02-28', '2019-03-01']


2025-05-13 05:58:07,792 INFO Request ID is 7a4547e8-cdea-4fa5-980a-5c03430a6029
2025-05-13 05:58:07,792 - INFO - Request ID is 7a4547e8-cdea-4fa5-980a-5c03430a6029
2025-05-13 05:58:07,935 INFO status has been updated to accepted
2025-05-13 05:58:07,935 - INFO - status has been updated to accepted
2025-05-13 05:58:16,312 INFO status has been updated to running
2025-05-13 05:58:16,312 - INFO - status has been updated to running
2025-05-13 05:58:29,104 INFO status has been updated to successful
2025-05-13 05:58:29,104 - INFO - status has been updated to successful
2025-05-13 05:58:29,188 - INFO - Downloading https://object-store.os-api.cci2.ecmwf.int:443/cci2-prod-cache-1/2025-05-10/3a76eae1a4406eb2f3a9c1761da77354.zip


ERS5 for ['2019-02-28', '2019-03-01'] is written to cubes/aux/era5_land/2019_02_28.zarr
ERS5 for ['2019-03-02', '2019-03-03'] is already written to cubes/aux/era5_land/2019_03_02.zarr
ERS5 for ['2019-03-04', '2019-03-05'] is already written to cubes/aux/era5_land/2019_03_04.zarr
ERS5 for ['2019-03-06', '2019-03-07'] is already written to cubes/aux/era5_land/2019_03_06.zarr
ERS5 for ['2019-03-08', '2019-03-09'] is already written to cubes/aux/era5_land/2019_03_08.zarr
ERS5 for ['2019-03-10', '2019-03-11'] is already written to cubes/aux/era5_land/2019_03_10.zarr
ERS5 for ['2019-03-12', '2019-03-13'] is already written to cubes/aux/era5_land/2019_03_12.zarr
ERS5 for ['2019-03-14', '2019-03-15'] is already written to cubes/aux/era5_land/2019_03_14.zarr
ERS5 for ['2019-03-16', '2019-03-17'] is already written to cubes/aux/era5_land/2019_03_16.zarr
ERS5 for ['2019-03-18', '2019-03-19'] is already written to cubes/aux/era5_land/2019_03_18.zarr
ERS5 for ['2019-03-20', '2019-03-21'] is already

xcube-cds version 1.0.0
2025-05-13 05:59:02,015 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 05:59:02,015 - INFO - [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-05-13 05:59:02,016 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 05:59:02,016 - WARNING - [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-05-13 05:59:02,172 INFO Request ID is 14caf8e7-c403-4a46-8701-6e1bd97

ERS5 for ['2019-09-04', '2019-09-05'] is written to cubes/aux/era5_land/2019_09_04.zarr
ERS5 for ['2019-09-06', '2019-09-07'] is already written to cubes/aux/era5_land/2019_09_06.zarr
ERS5 for ['2019-09-08', '2019-09-09'] is already written to cubes/aux/era5_land/2019_09_08.zarr
ERS5 for ['2019-09-10', '2019-09-11'] is already written to cubes/aux/era5_land/2019_09_10.zarr
ERS5 for ['2019-09-12', '2019-09-13'] is already written to cubes/aux/era5_land/2019_09_12.zarr
ERS5 for ['2019-09-14', '2019-09-15'] is already written to cubes/aux/era5_land/2019_09_14.zarr
ERS5 for ['2019-09-16', '2019-09-17'] is already written to cubes/aux/era5_land/2019_09_16.zarr
ERS5 for ['2019-09-18', '2019-09-19'] is already written to cubes/aux/era5_land/2019_09_18.zarr
ERS5 for ['2019-09-20', '2019-09-21'] is already written to cubes/aux/era5_land/2019_09_20.zarr
ERS5 for ['2019-09-22', '2019-09-23'] is already written to cubes/aux/era5_land/2019_09_22.zarr
ERS5 for ['2019-09-24', '2019-09-25'] is already

If the requested time range covers a month boundary, too much data is requested. (see https://github.com/xcube-dev/xcube-cds/issues/99) We will clean this in the following cell. 

In [5]:
for time_range in time_ranges:
    file_name = time_range[0].replace("-", "_")
    data_id = f"cubes/aux/era5_land/{file_name}.zarr"
    ds = store_team.open_data(data_id)
    ds_cut = ds.sel(time=slice(
        datetime.datetime.fromisoformat(time_range[0] + "T00:00:00"),
        datetime.datetime.fromisoformat(time_range[1] + "T23:59:59"),
    ))
    if ds_cut.time.size != ds.time.size:
        LOG.info(f"Error in time axis found for {data_id}. Dataset will be cut off.")
        data_id_mod = data_id.replace(".zarr", "_mod.zarr")
        ds_cut = chunk_dataset(ds_cut, chunk_sizes={"time": 24, "lat":125, "lon":200}, format_name="zarr")
        store_team.write_data(ds_cut, data_id_mod, replace=True)
        store_team.delete_data(data_id)
        ds_mod = store_team.open_data(data_id_mod)
        store_team.write_data(ds_mod, data_id, replace=True)
        store_team.delete_data(data_id_mod)

2025-05-13 10:29:17,569 - INFO - Error in time axis found for cubes/aux/era5_land/2017_01_31.zarr. Dataset will be cut off.
2025-05-13 10:29:52,804 - INFO - Error in time axis found for cubes/aux/era5_land/2017_02_28.zarr. Dataset will be cut off.
2025-05-13 10:30:37,322 - INFO - Error in time axis found for cubes/aux/era5_land/2017_05_31.zarr. Dataset will be cut off.
2025-05-13 10:31:12,639 - INFO - Error in time axis found for cubes/aux/era5_land/2017_06_30.zarr. Dataset will be cut off.
2025-05-13 10:31:53,346 - INFO - Error in time axis found for cubes/aux/era5_land/2017_08_31.zarr. Dataset will be cut off.
2025-05-13 10:32:28,496 - INFO - Error in time axis found for cubes/aux/era5_land/2017_09_30.zarr. Dataset will be cut off.
2025-05-13 10:33:17,718 - INFO - Error in time axis found for cubes/aux/era5_land/2018_02_28.zarr. Dataset will be cut off.
2025-05-13 10:34:00,119 - INFO - Error in time axis found for cubes/aux/era5_land/2018_05_31.zarr. Dataset will be cut off.
2025-05-

Rechunk dataset if they cover 3 days, oherwise we get problems when using zappend. 

In [9]:
for year in range(2017, 2025):
    data_id = f"cubes/aux/era5_land/{year}_12_29.zarr"
    if store_team.has_data(data_id):
        LOG.info(f"Change time chunking for {data_id}.")
        ds = store_team.open_data(data_id)
        data_id_mod = data_id.replace(".zarr", "_mod.zarr")
        ds_rechunk = chunk_dataset(ds, chunk_sizes={"time": 24, "lat":125, "lon":200}, format_name="zarr")
        store_team.write_data(ds_rechunk, data_id_mod, replace=True)
        store_team.delete_data(data_id)
        ds_mod = store_team.open_data(data_id_mod)
        store_team.write_data(ds_mod, data_id, replace=True)
        store_team.delete_data(data_id_mod)

In [4]:
data_ids = store_team.list_data_ids()
data_ids = [data_id for data_id in data_ids if "cubes/aux/era5_land/" in data_id]
data_ids

['cubes/aux/era5_land/2016_11_01.zarr',
 'cubes/aux/era5_land/2016_11_03.zarr',
 'cubes/aux/era5_land/2016_11_05.zarr',
 'cubes/aux/era5_land/2016_11_07.zarr',
 'cubes/aux/era5_land/2016_11_09.zarr',
 'cubes/aux/era5_land/2016_11_11.zarr',
 'cubes/aux/era5_land/2016_11_13.zarr',
 'cubes/aux/era5_land/2016_11_15.zarr',
 'cubes/aux/era5_land/2016_11_17.zarr',
 'cubes/aux/era5_land/2016_11_19.zarr',
 'cubes/aux/era5_land/2016_11_21.zarr',
 'cubes/aux/era5_land/2016_11_23.zarr',
 'cubes/aux/era5_land/2016_11_25.zarr',
 'cubes/aux/era5_land/2016_11_27.zarr',
 'cubes/aux/era5_land/2016_11_29.zarr',
 'cubes/aux/era5_land/2016_12_01.zarr',
 'cubes/aux/era5_land/2016_12_03.zarr',
 'cubes/aux/era5_land/2016_12_05.zarr',
 'cubes/aux/era5_land/2016_12_07.zarr',
 'cubes/aux/era5_land/2016_12_09.zarr',
 'cubes/aux/era5_land/2016_12_11.zarr',
 'cubes/aux/era5_land/2016_12_13.zarr',
 'cubes/aux/era5_land/2016_12_15.zarr',
 'cubes/aux/era5_land/2016_12_17.zarr',
 'cubes/aux/era5_land/2016_12_19.zarr',


In [7]:
# open dataset and check for nan values
for i, data_id in enumerate(data_ids):
    if i % 100 == 0:
        LOG.info(f"Checking {data_id}.")
    ds = store_team.open_data(data_id)
    for var in ds.data_vars:
        vals = ds[var].values
        nans = vals[np.isnan(vals)] 
        perc = nans.size / vals.size * 100 
        if perc  > 31.504:
            LOG.info(
                f"{data_id} has {perc}% nan"
                f"values for variable name {var}."
            )

2025-05-13 06:11:25,189 - INFO - Checking cubes/aux/era5_land/2016_11_01.zarr.
2025-05-13 06:20:20,635 - INFO - Checking cubes/aux/era5_land/2017_05_21.zarr.
2025-05-13 06:29:08,890 - INFO - Checking cubes/aux/era5_land/2017_12_07.zarr.
2025-05-13 06:37:47,203 - INFO - Checking cubes/aux/era5_land/2018_06_26.zarr.
2025-05-13 06:46:07,931 - INFO - Checking cubes/aux/era5_land/2019_01_13.zarr.
2025-05-13 06:54:26,353 - INFO - Checking cubes/aux/era5_land/2019_08_01.zarr.
2025-05-13 07:02:54,151 - INFO - Checking cubes/aux/era5_land/2020_02_18.zarr.
2025-05-13 07:12:11,511 - INFO - Checking cubes/aux/era5_land/2020_09_05.zarr.
2025-05-13 07:21:10,889 - INFO - Checking cubes/aux/era5_land/2021_03_24.zarr.
2025-05-13 07:30:11,761 - INFO - Checking cubes/aux/era5_land/2021_10_10.zarr.
2025-05-13 07:38:53,664 - INFO - Checking cubes/aux/era5_land/2022_04_29.zarr.
2025-05-13 07:47:04,158 - INFO - Checking cubes/aux/era5_land/2022_11_15.zarr.
2025-05-13 07:55:15,190 - INFO - Checking cubes/aux/

In [4]:
data_ids = store_team.list_data_ids()
data_ids = [data_id for data_id in data_ids if "cubes/aux/era5_land/" in data_id]
data_ids

['cubes/aux/era5_land/2016_11_01.zarr',
 'cubes/aux/era5_land/2016_11_03.zarr',
 'cubes/aux/era5_land/2016_11_05.zarr',
 'cubes/aux/era5_land/2016_11_07.zarr',
 'cubes/aux/era5_land/2016_11_09.zarr',
 'cubes/aux/era5_land/2016_11_11.zarr',
 'cubes/aux/era5_land/2016_11_13.zarr',
 'cubes/aux/era5_land/2016_11_15.zarr',
 'cubes/aux/era5_land/2016_11_17.zarr',
 'cubes/aux/era5_land/2016_11_19.zarr',
 'cubes/aux/era5_land/2016_11_21.zarr',
 'cubes/aux/era5_land/2016_11_23.zarr',
 'cubes/aux/era5_land/2016_11_25.zarr',
 'cubes/aux/era5_land/2016_11_27.zarr',
 'cubes/aux/era5_land/2016_11_29.zarr',
 'cubes/aux/era5_land/2016_12_01.zarr',
 'cubes/aux/era5_land/2016_12_03.zarr',
 'cubes/aux/era5_land/2016_12_05.zarr',
 'cubes/aux/era5_land/2016_12_07.zarr',
 'cubes/aux/era5_land/2016_12_09.zarr',
 'cubes/aux/era5_land/2016_12_11.zarr',
 'cubes/aux/era5_land/2016_12_13.zarr',
 'cubes/aux/era5_land/2016_12_15.zarr',
 'cubes/aux/era5_land/2016_12_17.zarr',
 'cubes/aux/era5_land/2016_12_19.zarr',


In [5]:
final_data_id = "cubes/aux/era5_land.zarr"
config = {
    "target_dir": f"s3://{os.environ['S3_USER_STORAGE_BUCKET']}/{final_data_id}",
    "target_storage_options": {
        "key": os.environ["S3_USER_STORAGE_KEY"],
        "secret": os.environ["S3_USER_STORAGE_SECRET"],
    },
    "slice_storage_options": {
        "key": os.environ["S3_USER_STORAGE_KEY"],
        "secret": os.environ["S3_USER_STORAGE_SECRET"],              
    }, 
    "force_new": True,
    "logging": True,
    "excluded_variables": ["expver", "number"]
}
zappend((f"s3://{os.environ['S3_USER_STORAGE_BUCKET']}/{data_id}" for data_id in data_ids), config=config)

2025-05-14 16:06:24,279 WARNING Setting 'force_new' is enabled. This will permanently delete existing targets (no rollback).
2025-05-14 16:06:24,279 - WARNING - Setting 'force_new' is enabled. This will permanently delete existing targets (no rollback).
2025-05-14 16:06:24,722 WARNING Permanently deleting s3://hub-deepesdl37/cubes/aux/era5_land.zarr
2025-05-14 16:06:24,722 - WARNING - Permanently deleting s3://hub-deepesdl37/cubes/aux/era5_land.zarr
2025-05-14 16:06:53,104 INFO Opening slice dataset from s3://hub-deepesdl37/cubes/aux/era5_land/2016_11_01.zarr
2025-05-14 16:06:53,104 - INFO - Opening slice dataset from s3://hub-deepesdl37/cubes/aux/era5_land/2016_11_01.zarr
2025-05-14 16:06:53,392 INFO Creating target dataset s3://hub-deepesdl37/cubes/aux/era5_land.zarr
2025-05-14 16:06:53,392 - INFO - Creating target dataset s3://hub-deepesdl37/cubes/aux/era5_land.zarr
/home/conda/deepesdl37/bb81b95c-1746601496-15-cubegen/lib/python3.13/site-packages/zappend/processor.py:151: FutureWar

1488

In [4]:
final_data_id = "cubes/aux/era5_land.zarr"
ds = store_team.open_data(final_data_id)
ds

<xarray.Dataset> Size: 802GB
Dimensions:  (time: 71592, lat: 250, lon: 400)
Coordinates:
  * lat      (lat) float64 2kB 64.95 64.85 64.75 64.65 ... 40.25 40.15 40.05
  * lon      (lon) float64 3kB -4.95 -4.85 -4.75 -4.65 ... 34.75 34.85 34.95
  * time     (time) datetime64[ns] 573kB 2016-11-01 ... 2024-12-31T23:00:00
Data variables: (12/28)
    d2m      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    e        (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    fal      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    lai_hv   (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    lai_lv   (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    pev      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    ...       ...
    swvl3    (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    swvl4    (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    t2m      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    tp       (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    u10      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    v10      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-05-07T12:56 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [8]:
# open dataset and check for nan values
final_data_id = "cubes/aux/era5_land.zarr"
ds = store_team.open_data(final_data_id)
for var in list(ds.keys())[26:]:
    LOG.info(f"Checking variable {var}.")
    for i in range(0, ds.sizes["time"], 24):
        day_slice = ds[var].isel(time=slice(i, i+24))
        vals = day_slice.values
        nans = vals[da.isnan(vals)] 
        perc = nans.size / vals.size * 100 
        if perc  > 31.504:
            LOG.info(
                f"{final_data_id} has {perc}% nan"
                f"values for variable name {var}"
                f"for index {i} to {i+24}."
            )

2025-05-17 04:04:30,115 - INFO - Checking variable u10.
2025-05-17 04:10:30,670 - INFO - Checking variable v10.


In [5]:
final_data_id = "cubes/aux/era5_land.zarr"
ds = store_team.open_data(final_data_id)
ds

<xarray.Dataset> Size: 802GB
Dimensions:  (time: 71592, lat: 250, lon: 400)
Coordinates:
  * lat      (lat) float64 2kB 64.95 64.85 64.75 64.65 ... 40.25 40.15 40.05
  * lon      (lon) float64 3kB -4.95 -4.85 -4.75 -4.65 ... 34.75 34.85 34.95
  * time     (time) datetime64[ns] 573kB 2016-11-01 ... 2024-12-31T23:00:00
Data variables: (12/28)
    d2m      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    e        (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    fal      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    lai_hv   (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    lai_lv   (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    pev      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    ...       ...
    swvl3    (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    swvl4    (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    t2m      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    tp       (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    u10      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
    v10      (time, lat, lon) float32 29GB dask.array<chunksize=(24, 125, 200), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-05-07T12:56 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [6]:
chunk_sizes = dict(time=24*365, lat=10, lon=10)
ds = chunk_dataset(ds, chunk_sizes=chunk_sizes, format_name="zarr")
ds

<xarray.Dataset> Size: 802GB
Dimensions:  (time: 71592, lat: 250, lon: 400)
Coordinates:
  * lat      (lat) float64 2kB 64.95 64.85 64.75 64.65 ... 40.25 40.15 40.05
  * lon      (lon) float64 3kB -4.95 -4.85 -4.75 -4.65 ... 34.75 34.85 34.95
  * time     (time) datetime64[ns] 573kB 2016-11-01 ... 2024-12-31T23:00:00
Data variables: (12/28)
    d2m      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    e        (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    fal      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    lai_hv   (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    lai_lv   (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    pev      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    ...       ...
    swvl3    (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    swvl4    (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    t2m      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    tp       (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    u10      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
    v10      (time, lat, lon) float32 29GB dask.array<chunksize=(8760, 10, 10), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-05-07T12:56 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [7]:
new_data_id = "cubes/aux/era5_land_time_optimized.zarr"
store_team.write_data(ds, new_data_id, replace=True)

'cubes/aux/era5_land_time_optimized.zarr'